In [4]:
import os
import numpy as np
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Embedding, Input, Dropout, Flatten
from keras.utils import plot_model
from keras.initializers import RandomUniform
from keras import preprocessing
import matplotlib.pyplot as plt
from keras.datasets import reuters

## Einführung
Diese Templates sollen der Unterstützung dienen. Sie dürfen sinnvoll abgeändert werden, z.B. Parameter zu Funktionen hinzugefügt oder zusätliche eigene Funktionen geschrieben werden.
Als Abgabe dient das ausgeführte, d.h. mit Output, gespeicherte Notebook.

## Aufgabe 1a

In [ ]:
class WNDWDATA(object):
    
    def __init__(self):
        """Initialize the data object"""
        self.window_size = 5        
                 
    def readFile(self, filename):
        """ read file
        return format : [[Word, Label]]
        [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]"""
        f = open(filename)
        sentences = []
        
        # YOUR CODE HERE
        
        return sentences

    def loadData(self):
        """Load data"""
        #call readFile for training, validation (, and later testing) data
        self.trainSentences = self.readFile("../../ralf/data/coNLL/eng.train")
        self.devSentences = self.readFile("../../ralf/data/coNLL/eng.testa")
        #self.testSentences = self.readFile("../../ralf/data/coNLL/eng.testb")

    def createDictionary(self):
        
        # create mapping for labels (label->label_id)
        # create mapping for words (word->wordID)
        # create one special wordID for "PADDING_TOKEN" and one for "UNKNOWN_TOKEN"
        self.label2Idx = {}
        self.word2Idx = {}
    
        # YOUR CODE HERE        


    def loadEmbeddings(self):
        """Create word-level embeddings"""
        # read GLoVE word embeddings
        self.wordEmbeddings1 = []
        
        fEmbeddings = open("../../ralf/data/glove/glove.6B.50d.txt", encoding="utf-8")

        # loop through each word in the loaded embeddings
        # keep only the embeddings for words that appear in your data
        # make sure to lower case your words!
      
        
        # YOUR CODE HERE

        # read stored own embeddings
        self.wordEmbeddings2 = []
        
        
        # YOUR CODE HERE
    
    
        print("Embeddings loaded.")
        
    # replace strings with ids (wordIDs and labelIDs) in the data
    # return format: [[wordIDs][label]]
    # [[4,7,2][0,0,1]]
    def createInputMatrix(self, sents):
        unknownIdx = self.word2Idx['UNKNOWN_TOKEN']

        sentences = []
   
    
        # YOUR CODE HERE
    
    
        return sentences
    
    # returns samples of shape
    # [[wordIDs (1 + 2 * window size)]], [label]
    # [[0,0,4,6,2]] [0]
    # we want to label the center word of our window, that means padding for the first and last words in a sentence!
    def createSamples(self, sents):
        padding = self.word2Idx["PADDING_TOKEN"]
        x = []
        y = []
     
    
        # YOUR CODE HERE
    
    
        return np.array(x),np.array(y)
    
    def prepareData(self):

        # format: [[wordindices],[labelindices]]
        self.trainset = self.createMatrices(self.trainSentences)
        self.valset = self.createMatrices(self.devSentences)
        # self.testset = self.createMatrices(self.testSentences)

        # format: [[wordindices]],[labelindex]
        self.x_train, self.y_train = self.createSamples(self.trainset)
        self.x_val, self.y_val = self.createSamples(self.valset)
        # self.x_test, self.y_test = self.createSamples(self.testset)

In [ ]:
wndwData = WNDWDATA()
wndwData.loadData()
wndwData.createDictionary()
wndwData.loadEmbeddings()
wnDwData.prepareData()

## Aufgabe 1b

In [ ]:
class WNDW(object):
    
    def __init__(self, DATA):
        """Initialize the model"""
        self.epochs = 5
        self.dropout = 0.5
        self.data = DATA
        
    def predictLabel(self, x_testset, y_testset):
        """Tag unseen data with numerical values (label IDs)"""
        correctLabels = []
        predLabels = []
        
        
        # YOUR CODE HERE
        
        
        return np.array(predLabels), np.array(correctLabels)
    
    def buildModel(self, EMBTYPE):
        """Build the model"""
        # Build a Sequential() model with the layers you need; add dropout if you want.
        
        if(EMBTYPE==1):
        
        # YOUR CODE HERE
        
        else if(EMBTYPE==2):
        
        # YOUR CODE HERE

        
        # Compile the model (model.compile)
        
        
        # YOUR CODE HERE
        
        
        #Plot the network architecture
        plot_model(self.model, to_file='model.png')
        print("Model built. Saved model.png\n")
        
    def train(self):
        """Train the model"""
        # Train the model (model.fit) and store the output in model.history
        
        
        # YOUR CODE HERE
        
        
        print("Training finished.")
        # (optional) save model to file if you don't want to train from scratch everytime
        self.modelName = "{}_{}_{}".format(self.epochs, self.data.window_size, self.dropout)
        
        modelName = self.modelName + ".h5"
        self.model.save(modelName)
        print("Model weights saved.")
        

    def plotTraining(self):
        """Plot graphs"""
        # Load the training statistics (model.history)
        # Plot training loss and accuracy
       
        
        # YOUR CODE HERE
        
        
        plt.plot(epochs, loss_values, 'bo', label='Training loss')
        plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        plt.clf()

        # Plot validation loss and accuracy
        
        
        # YOUR CODE HERE

        
        plt.plot(epochs, acc_values, 'bo', label='Training acc')
        plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()
        
    def evaluate(self):
        """Test the model"""
        # Predict labels for the test data
        predLab, corLab = wndw.predictLabel(data.x_test, data.y_test)
        # Compare to gold standard/ground truth
        # First on token level, i.e. for each individual word
        # Print Precision, Recall, and F1 for all labels not O
        
        
        # YOUR CODE HERE
        
        
        # Output confusion matrix predicted label vs. true label token level
       
    
        # YOUR CODE HERE
        
        
        # Second on entity level, i.e. for each (multi-word) entity. E.g. ...New York... is only correctly labeled if both "New" and "York" were labeled as I-LOC (or B-LOC I-LOC) 
        # Print Precision, Recall, and F1 for all labels not O
        
        
        # YOUR CODE HERE
        
        
        # Output confusion matrix predicted label vs. true label entity level
        
        
        # YOUR CODE HERE
        

    print("Class initialised.")

In [ ]:
"""Construct and run the model with pretrained Glove Embeddings"""
wndw1 = WNDW(wndwData)
wndw1.buildModel(1)
wndw1.train()
wndw1.plotTraining()
wndw1.evaluate()

In [ ]:
"""Construct and run the model with task-specific Embeddings"""
wndw2 = WNDW(wndwData)
wndw2.buildModel(2)
wndw2.train()
wndw2.plotTraining()
wndw2.evaluate()

## Aufgabe 1c

In [ ]:
class RNNDATA(object):
    
    def __init__(self):
        """Initialize the data object"""
        self.window_size = 5        
                 
    def readFile(self, filename):
        """ read file
        return format : [[Word, Label]]
        [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]"""
        f = open(filename)
        sentences = []
        
        # YOUR CODE HERE
        
        return sentences

    def loadData(self):
        """Load data"""
        # call readFile for training, validation (, and later testing) data
        self.trainSentences = self.readFile("../../ralf/data/coNLL/eng.train")
        self.devSentences = self.readFile("../../ralf/data/coNLL/eng.testa")
        # self.testSentences = self.readFile("../../ralf/data/coNLL/eng.testb")

    def createDictionary(self):
        
        # create mapping for labels (label->label_id)
        # create mapping for words (word->wordID)
        # create one special wordID for "PADDING_TOKEN" and one for "UNKNOWN_TOKEN"
        self.label2Idx = {}
        self.word2Idx = {}
    
        # YOUR CODE HERE        


    def loadEmbeddings(self):
        """Create word-level embeddings"""
        # read GLoVE word embeddings
        self.wordEmbeddings = []
        
        fEmbeddings = open("../../ralf/data/glove/glove.6B.50d.txt", encoding="utf-8")

        # loop through each word in the loaded embeddings
        # keep only the embeddings for words that appear in your data
        # make sure to lower case your words!
      
        
        # YOUR CODE HERE
        
    
        print("Embeddings loaded.")
        
    # replace strings with ids (wordIDs and labelIDs) in the data
    # return format: [[wordIDs][label]]
    # [[4,7,2][0,0,1]]
    def createInputMatrix(self, sents):
        unknownIdx = self.word2Idx['UNKNOWN_TOKEN']

        sentences = []
   
    
        # YOUR CODE HERE
    
    
        return sentences
    
    # returns samples of shape
    # [[wordIDs (1 + 2 * window size)]], [label]
    # [[0,0,4,6,2]] [0]
    # we want the label the center word of our window, that means padding for the first and last words in a sentence!
    def createSamples(self, sents):
        padding = self.word2Idx["PADDING_TOKEN"]
        x = []
        y = []
     
    
        # YOUR CODE HERE
    
    
        return np.array(x),np.array(y)
    
    def prepareData(self):

        # format: [[wordindices],[labelindices]]
        self.trainset = self.createMatrices(self.trainSentences)
        self.valset = self.createMatrices(self.devSentences)
        #self.testset = self.createMatrices(self.testSentences)

        # format: [[wordindices]],[labelindex]
        self.x_train, self.y_train = self.createSamples(self.trainset)
        self.x_val, self.y_val = self.createSamples(self.valset)
        #self.x_test, self.y_test = self.createSamples(self.testset)

In [ ]:
rnnData = RNNDATA()
rnnData.loadData()
rnnData.createDictionary()
rnnData.loadEmbeddings()
rnnData.prepareData()

In [ ]:
class RNN(object):
    
    def __init__(self, DATA):
        """Initialize the model"""
        self.epochs = EPOCHS
        self.dropout = DROPOUT
        self.data = DATA
        
    def predictLabel(self, x_testset, y_testset):
        """Tag unseen data with numerical values (label IDs)"""
        correctLabels = []
        predLabels = []
        
        
        # YOUR CODE HERE
        
        
        return np.array(predLabels), np.array(correctLabels)
    
    def buildModel(self):
        """Build the model"""
        # Build a Sequential() model with the layers you need; add dropout if you want.
        
        
        # YOUR CODE HERE

        
        # Compile the model (model.compile)
        
        
        # YOUR CODE HERE
        
        #Plot the network architecture
        plot_model(self.model, to_file='model.png')
        print("Model built. Saved model.png\n")
        
    def train(self):
        """Train the model"""
        # Train the model (model.fit) and store the output in model.history
        
        
        # YOUR CODE HERE
        
        
        print("Training finished.")
        # (optional) save model to file if you don't want to train from scratch everytime
        self.modelName = "{}_{}_{}".format(self.epochs, self.data.window_size, self.dropout)
        
        modelName = self.modelName + ".h5"
        self.model.save(modelName)
        print("Model weights saved.")
        

    def plotTraining(self):
        """Plot graphs"""
        # Load the training statistics (model.history)
        # Plot training loss and accuracy
        
        
        # YOUR CODE HERE
        
        
        plt.plot(epochs, loss_values, 'bo', label='Training loss')
        plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        plt.clf()

        # Plot validation loss and accuracy
        
        
        # YOUR CODE HERE

        
        plt.plot(epochs, acc_values, 'bo', label='Training acc')
        plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()
        
    def evaluate(self):
        """Test the model"""
        # Predict labels for the test data
        predLab, corLab = wndw.predictLabel(data.x_test, data.y_test)
        # Compare to gold standard/ground truth
        # First on token level, i.e. for each individual word
        # Print Precision, Recall, and F1 for all labels not O
        
        
        # YOUR CODE HERE
        
        
        # Output confusion matrix predicted label vs. true label token level
       
    
        # YOUR CODE HERE
        
        
        # Second on entity level, i.e. for each (multi-word) entity. E.g. ...New York... is only correctly labeled if both "New" and "York" were labeled as I-LOC (or B-LOC I-LOC) 
        # Print Precision, Recall, and F1 for all labels not O
        
        
        # YOUR CODE HERE
        
        
        # Output confusion matrix predicted label vs. true label entity level
        
        
        # YOUR CODE HERE
        

    print("Class initialised.")

In [ ]:
"""Construct and run model"""
rnn = RNN(rnnData)
rnn.buildModel()
rnn.train()
rnn.plotTraining()
rnn.evaluate()